<a href="https://colab.research.google.com/github/RainaVardhan/Data-Science-Systems-HW-Labs/blob/main/RainaVardhan/BuildingAChatbot/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [258]:
!pip install tensorflow

In [259]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np

In [260]:
questions = [
    'Hello',
    'How are you?',
    'What is your name?',
    'What is TensorFlow?',
    'Do you like Python?'
]

answers = [
    'Hi! How can I help you today?',
    'I am doing fine.',
    'I do not have a name.',
    'TensorFlow is a machine learning models.',
    'Yes, Python is one of my favorite programming languages.'
]

In [261]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

In [262]:
max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

In [263]:
vocab_size = len(tokenizer.word_index) + 1

In [264]:
embedding_dim = 256
units = 1024

In [265]:
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

In [266]:
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [267]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_30 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 input_31 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_29 (Embedding)    (None, None, 256)            8704      ['input_30[0][0]']            
                                                                                                  
 embedding_30 (Embedding)    (None, None, 256)            8704      ['input_31[0][0]']            
                                                                                           

In [268]:
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1
model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=100)

Epoch 1/100
3/3 [==============================] - 6s 144ms/step - loss: 3.4223 - accuracy: 0.2444
Epoch 2/100
3/3 [==============================] - 0s 22ms/step - loss: 3.2890 - accuracy: 0.3111
Epoch 3/100
3/3 [==============================] - 0s 63ms/step - loss: 3.0065 - accuracy: 0.3333
Epoch 4/100
3/3 [==============================] - 0s 127ms/step - loss: 2.7817 - accuracy: 0.3333
Epoch 5/100
3/3 [==============================] - 0s 22ms/step - loss: 2.7719 - accuracy: 0.3333
Epoch 6/100
3/3 [==============================] - 0s 59ms/step - loss: 2.9101 - accuracy: 0.3333
Epoch 7/100
3/3 [==============================] - 0s 16ms/step - loss: 2.6678 - accuracy: 0.3333
Epoch 8/100
3/3 [==============================] - 0s 15ms/step - loss: 2.6505 - accuracy: 0.3333
Epoch 9/100
3/3 [==============================] - 0s 16ms/step - loss: 2.5867 - accuracy: 0.2667
Epoch 10/100
3/3 [==============================] - 0s 15ms/step - loss: 2.5661 - accuracy: 0.2889
Epoch 11/100
3/3 

In [269]:
# # Step 5: Implement the chat function
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # end token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

In [270]:
import requests
def get_weather(city):
    api_key = "bc064a8a4f0e13125929ea154b6fcb10"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=imperial"
    response = requests.get(url)
    x = response.json()
    if x["cod"] != "404":
        # y = x["main"]
        current_temperature = x["main"]["temp"]
        current_pressure = x["main"]["pressure"]
        current_humidity = x["main"]["humidity"]
        # z = x["weather"]
        weather_description = x["weather"][0]["description"]
        return f"The weather in {city} is:  \n Temperature: {str(current_temperature)}\n Atmospheric pressure: {str(current_pressure)}\n Humidity: {str(current_humidity)}\n Description: {str(weather_description)}"
    else:
        return "Sorry, I could not find any weather data on the city."

In [271]:
def chat_with_bot(input_text):
    if 'weather' in input_text.lower():
        city_name = input_text.split('weather in ')[1]
        return get_weather(city_name)
    else:
        input_sequence = preprocess_input_text(input_text)
        response_sequence = generate_response(input_sequence)
        response_text = sequence_to_text(response_sequence[0])
        return response_text

input_text = "Do you like Python?"
print(f"You: {input_text}")
print(f"Bot: {chat_with_bot(input_text)}")
input_text = "What is the weather in London"
print(f"You: {input_text}")
print(f"Bot: {chat_with_bot(input_text)}")

You: Do you like Python?
1/1 [==============================] - 0s 22ms/step
Bot: yes python one of my favorite programming
You: What is the weather in London
Bot: The weather in London is:  
 Temperature: 47.5
 Atmospheric pressure: 1009
 Humidity: 82
 Description: scattered clouds


In [275]:
# Interactive chat with the bot
print("Start chatting with the bot! Type 'Goodbye' to exit.")
while True:
    user_input = input("You: ")
    if ((user_input.lower() == 'goodbye') or (user_input.lower() == 'i am done')):
        print('Goodbye!')
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")

Start chatting with the bot! Type 'Goodbye' to exit.
You: Hello
1/1 [==============================] - 0s 21ms/step
Bot: hi how can i help
You: What is your name
1/1 [==============================] - 0s 20ms/step
Bot: i do not have a name
You: What is TensorFlow
1/1 [==============================] - 0s 20ms/step
Bot: tensorflow a machine learning models
You: Do you like Python
1/1 [==============================] - 0s 21ms/step
Bot: yes python one of my favorite programming
You: What is the weather in Charlottesville
Bot: The weather in Charlottesville is:  
 Temperature: 70.32
 Atmospheric pressure: 1009
 Humidity: 35
 Description: clear sky
You: What is the weather in nonsense
Bot: Sorry, I could not find any weather data on the city.
You: I am done
Goodbye!
